# 「單一」股票最佳擇時(Timing)因子選擇

In [ ]:
"""
因子優化模組使用範例
"""
from alanq.optimization import ParameterSpace, FactorOptimizer
from alanq.factors.timing import BreakoutBuyFactor, BreakdownSellFactor, AtrStopSellFactor
from alanq.data import StockDataManager

def example_optimization():
    """範例：單股票因子優化"""
    
    # 1. 準備資料
    print("正在下載資料...")
    data_manager = StockDataManager(["TSLA"], start_date="2020-01-01")
    df = data_manager.get_kl_pd("TSLA")
    
    # 2. 定義參數空間
    print("定義參數空間...")
    param_space = ParameterSpace()
    
    # 添加買入因子
    param_space.add_buy_factor(
        BreakoutBuyFactor,
        {'xd': [20, 30, 40, 50, 60], 'skip_days': [10, 15, 20]}
    )
    
    # 添加賣出因子
    param_space.add_sell_factor(
        BreakdownSellFactor,
        {'xd': [10, 15, 20, 25]}
    )
    
    param_space.add_sell_factor(
        AtrStopSellFactor,
        {'atr_n': [1.0, 1.5, 2.0, 2.5]}
    )
    
    # 查看總組合數
    total_combos = param_space.get_total_combinations()
    print(f"總組合數: {total_combos}")
    
    # 3. 執行優化
    print("\n開始優化...")
    # 定義自定義權重
    custom_weights = {
        '策略_總報酬率': 1,      # 30% 權重
        '策略_Sharpe': 1,       # 25% 權重
        '策略_最大回撤': 1,      # 20% 權重
        '勝率': 1,              # 15% 權重
        '盈虧比': 1,             # 10% 權重
        # 其他指標權重為 0（不考慮）
    }

    # 使用自定義權重進行優化
    optimizer = FactorOptimizer(
        df=df,
        parameter_space=param_space,
        metric_weights=custom_weights,  # 傳入自定義權重
        initial_capital=1_000_000,
        show_progress=True
    )
    # optimizer = FactorOptimizer(
    #     df=df,
    #     parameter_space=param_space,
    #     initial_capital=1_000_000,
    #     show_progress=True
    # )
    
    best_config, results_df = optimizer.optimize()
    
    # 4. 查看結果
    if best_config:
        print(f"\n{'='*60}")
        print("最佳配置:")
        print(f"{'='*60}")
        print(f"買入因子: {best_config['buy_factors']}")
        print(f"賣出因子: {best_config['sell_factors']}")
        print(f"總得分: {best_config['總得分']:.2f}")
        
        print(f"\n{'='*60}")
        print("前 10 名結果:")
        print(f"{'='*60}")
        top_results = optimizer.get_top_n(10)
        display_cols = ['組合編號', '總得分', '策略_總報酬率', '策略_Sharpe', 
                       '策略_最大回撤', '勝率', '盈虧比']
        available_cols = [col for col in display_cols if col in top_results.columns]
        print(top_results[available_cols].to_string())
    else:
        print("優化失敗，沒有找到有效結果")
    
    return best_config, results_df

if __name__ == "__main__":
    best_config, results_df = example_optimization()

正在下載資料...
正在使用 yfinance 下載 1 檔股票資料...
------------------------------
已成功下載 1 檔股票資料
  - TSLA: 1481 筆資料，日期範圍 2020-01-02 至 2025-11-20
------------------------------
定義參數空間...
總組合數: 240

開始優化...
總共需要測試 240 種組合...


收集指標數據: 100%|██████████| 240/240 [00:25<00:00,  9.50it/s]


計算統計分佈並標準化得分...
使用的歸一化權重（總和為 1.0）:
  - 策略_總報酬率: 0.2000
  - 策略_Sharpe: 0.2000
  - 策略_最大回撤: 0.2000
  - 勝率: 0.2000
  - 盈虧比: 0.2000

最佳配置:
買入因子: [{'class': <class 'alanq.factors.timing.breakout_factor.BreakoutBuyFactor'>, 'xd': 20, 'skip_days': 10}]
賣出因子: [{'class': <class 'alanq.factors.timing.breakout_factor.BreakdownSellFactor'>, 'xd': 10}, {'class': <class 'alanq.factors.timing.atr_stop_sell_factor.AtrStopSellFactor'>, 'atr_n': 1.0}]
總得分: 58.97

前 10 名結果:
          總得分    策略_總報酬率  策略_Sharpe   策略_最大回撤
0   58.974214  13.877063   1.103083 -0.393893
1   58.974214  13.877063   1.103083 -0.393893
2   58.974214  13.877063   1.103083 -0.393893
3   58.974214  13.877063   1.103083 -0.393893
16  58.974214  13.877063   1.103083 -0.393893
17  58.974214  13.877063   1.103083 -0.393893
18  58.974214  13.877063   1.103083 -0.393893
19  58.974214  13.877063   1.103083 -0.393893
32  58.974214  13.877063   1.103083 -0.393893
33  58.974214  13.877063   1.103083 -0.393893


In [7]:
results_df

,總得分,買入因子,賣出因子,策略_總報酬率_得分,策略_年化報酬率_得分,策略_Sharpe_得分,策略_最大回撤_得分,策略_年化波動率_得分,勝率_得分,盈虧比_得分,...,profit_loss_ratio,total_profit,total_loss,net_profit,max_single_profit,max_single_loss,avg_holding_days,avg_return,max_consecutive_wins,max_consecutive_losses
0,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.900855e+07,6.131779e+06,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
1,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.900855e+07,6.131779e+06,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
2,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.900855e+07,6.131779e+06,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
3,58.974214,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,20.000000,0,20.000000,4.934675,0,6.218202,7.821337,...,2.893340,1.900855e+07,6.131779e+06,1.287678e+07,4.139933e+06,-1.113653e+06,27.896552,0.117906,4,4
4,44.475208,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,12.500439,0,14.951704,6.930613,0,6.314511,3.777941,...,2.088549,1.480446e+07,6.582082e+06,8.222383e+06,2.305038e+06,-8.197825e+05,36.259259,0.122823,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,42.960120,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,5.240043,0,11.210351,9.023386,0,17.486339,0.000000,...,1.336595,5.937309e+06,2.221058e+06,3.716251e+06,1.513443e+06,-9.460377e+05,61.833333,0.171668,4,3
236,56.402367,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,8.204530,0,13.511036,13.207720,0,20.000000,1.479082,...,1.630989,7.536413e+06,1.980327e+06,5.556086e+06,1.644835e+06,-1.234579e+06,84.700000,0.270484,4,3
237,56.402367,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,8.204530,0,13.511036,13.207720,0,20.000000,1.479082,...,1.630989,7.536413e+06,1.980327e+06,5.556086e+06,1.644835e+06,-1.234579e+06,84.700000,0.270484,4,3
238,56.402367,[{'class': <class 'alanq.factors.timing.breako...,[{'class': <class 'alanq.factors.timing.breako...,8.204530,0,13.511036,13.207720,0,20.000000,1.479082,...,1.630989,7.536413e+06,1.980327e+06,5.556086e+06,1.644835e+06,-1.234579e+06,84.700000,0.270484,4,3


# 多股投資組合最佳擇時(Timing)因子選擇
* 一般先用擇股(Selection)模組選出股票，再進行擇時